In this notebook, we want to summarize the data and make sure that it is behaving as we expect based on previous literature. Here, we want to only use the outcome for tumor_cell_evaluation

Tumor score is defined by UCSF pathologists in the following way: <br>

0: neuropil without tumor <br>
1: infiltrating tumor margin containing a detectable but not abundant number of tumor cells <br>
2: more cellular infiltrated zone <br>
3: highly cellular tumor <br>

In [2]:
recgli = read.csv("../../ParseData/8thParse_researchPath_withInVivo.csv") 

In [9]:
options(repr.matrix.max.rows=600, repr.matrix.max.cols=200)

#### In this notebook, we want to find: <br>
1. perfusion params vs. TS using <br>
    a. No separation into ROIs <br>
    b. Visual comparison of perfusion params in separate regions <br>
    c. Use GLMM associate perfusion parameters with TS, no separation <br>
    d. Use GEE to associate perfusion parameters with TS, no separation <br>
    e. Use GLMM to associate perfusion parameters with TS, separating into CEL & NEL <br>
    f. Use GEE to associate perfusion parameters with TS, separating into CEL & NEL <br>
2. Combining perf params vs. TS using GEE 
3. Using in_CEL as a covariate in perf params using GEE 

## 0. Prep the data

In [10]:
table(recgli$include_perf)
table(recgli$perf_quant)
dim(recgli)
119+272
recgli[which(recgli$include_perf ==0), ]


  0   1 
119 272 


  0   1   2 
272  29   1 

[1] 391  73

[1] 391

,b_number,t_number,sf_number,current_scan_date,current_surgery_date,current_hist_grade,current_hist_type,roi.label,tumor_cell_evaluation,necrosis,f8_delicate,f8_simple,f8_complex,mib_1,bx_pure_treatment_effect,nfse,nfl,nt1c,nt1v,nadc.1,nfa.1,nev1.1,nev2.1,nev3.1,nevrad.1,nadc.2,nfa.2,nev1.2,nev2.2,nev3.2,nevrad.2,cbvn_nlin,phn_nlin,phn_npar,recovn_npar,cni,ccri,crni,ncho,ncre,nnaa,nlip,laclip,nlac,olddata,newdata,comments,notes.,imaging_code,perf_quant,spec_quant,include_anat,include_diffu1000,include_diffu2000,include_diffu_all,include_perf,include_spec,include_one_advanced_mod,include_all_advanced_mods,X.CEL,X.NEL,X.NEC,sum,in_CEL,in_T2all,in_NEL,in_NEC,in_ROI,desired_hist,multnom_out,no_ffpe,waiting_on_path,rhgg_txe_analysis
13,2793,7584,8842,7/28/11,7/28/11,Grade IV,Glioblastoma,15B44,2,1,2,1,0,9.42,f,2.66,1.01,1.55,0.63,0.7200000,0.5000000,0.6300000,0.7000000,0.8900000,0.7900000,NA,1.28,2.67,2.31,2.16,2.30,NA,0,NA,0,NA,NA,NA,NA,NA,NA,NA,NA,NA,1,0,,NA,NA,1,NA,1,1,1,1,0,0,1,0,22.50,0.00,NA,22.50,1,0,0,0,1,1,rHGG,0,0,1
18,963,6069,7984,3/24/09,3/25/09,Grade IV,Glioblastoma,18B24,3,0,2,3,0,26.85,f,3.20,1.48,0.98,NA,2.0400000,0.7600000,1.8800000,2.1800000,2.1400000,2.2100000,NA,NA,NA,NA,NA,NA,NA,0,NA,0,NA,NA,NA,NA,NA,NA,NA,NA,NA,1,0,,NA,NA,1,NA,1,1,0,1,0,0,1,0,NA,NA,NA,NA,0,1,1,0,1,1,rHGG,0,0,1
20,2563,5983,7949,2/8/09,2/9/09,Grade IV,Glioblastoma,19B23,0,0,3,0,0,1.33,t,1.04,1.47,0.77,0.96,1.7000000,1.1000000,1.6500000,1.7000000,1.8300000,1.7700000,NA,NA,NA,NA,NA,NA,NA,t5983_19B23_outside_perfusion,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,1,0,,NA,NA,NA,NA,1,1,0,1,0,0,1,0,0.00,36.36,NA,36.36,0,1,1,0,1,1,TxE,0,0,1
25,2384,5603,7759,8/3/08,8/4/08,Grade III,Astrocytoma,21B18,2,0,2,2,0,5.02,f,1.86,1.38,1.63,0.67,1.4700000,0.5400000,1.3000000,1.4800000,1.8200000,1.6400000,NA,NA,NA,NA,NA,NA,NA,0,NA,0,NA,NA,NA,NA,NA,NA,NA,NA,NA,1,0,,NA,NA,1,NA,1,1,0,1,0,0,1,0,100.00,0.00,NA,100.00,1,1,0,0,1,1,rHGG,0,0,1
26,2384,5603,7759,8/3/08,8/4/08,Grade III,Astrocytoma,22B18,2,0,2,0,0,1.43,f,1.92,1.24,1.44,0.59,0.9100000,0.3600000,0.8100000,0.9200000,1.1100000,1.0100000,NA,NA,NA,NA,NA,NA,NA,0,NA,0,NA,NA,NA,NA,NA,NA,NA,NA,NA,1,0,,NA,NA,1,NA,1,1,0,1,0,0,1,0,66.67,14.29,NA,80.96,1,0,0,0,1,1,rHGG,0,0,1
27,2218,5656,7760,8/3/08,8/4/08,Grade IV,Glioblastoma,23B18,3,0,1,2,0,22.01,f,1.79,1.50,2.72,1.77,1.5600000,0.9100000,1.4100000,1.5100000,1.8200000,1.7100000,NA,NA,NA,NA,NA,NA,NA,perf_align-nonexist,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,1,0,,NA,NA,NA,NA,1,1,0,1,0,0,1,0,0.00,80.77,NA,80.77,1,1,1,0,1,1,rHGG,0,0,1
28,2089,5766,7835,10/19/08,10/20/08,Grade IV,Glioblastoma,23B20,3,1,0,3,0,24.11,f,1.92,1.49,1.39,0.85,1.1700000,0.8200000,1.0100000,1.2200000,1.3000000,1.2800000,NA,NA,NA,NA,NA,NA,NA,perf_align-nonexist,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,1,0,,NA,NA,NA,NA,1,1,0,1,0,0,1,0,53.85,19.23,11.54,84.62,1,1,0,0,1,1,rHGG,0,0,1
29,3171,7553,8804,6/24/11,6/27/11,Treatment Effect,Treatment Effect,25B43,0,0,0,0,0,0.00,t,0.66,0.54,1.11,0.87,0.8500000,1.7500000,1.0400000,0.9000000,0.6000000,0.8000000,0.67,1.53,0.87,0.55,0.32,0.46,NA,0,NA,0,NA,NA,NA,NA,NA,NA,NA,NA,NA,1,0,90% BV wall,NA,NA,1,NA,1,1,1,1,0,0,1,0,NA,NA,NA,NA,0,0,0,0,0,1,TxE,0,0,1
30,2192,8070,7540,11/18/07,11/19/07,Grade IV,Glioblastoma,26B13,2,0,3,2,0,8.11,f,1.44,0.99,0.86,0.84,1.2300000,1.0400000,1.1300000,1.1700000,1.2800000,1.2700000,NA,NA,NA,NA,NA,NA,NA,0,NA,0,NA,NA,NA,NA,NA,NA,NA,NA,NA,1,0,"necrosis on H&E<5%; PV tumor cells, on deeper sections ~20% of tissue is Tumor score=3",NA,NA,1,NA,1,1,0,1,0,0,1,0,0.00,NA,NA,0.00,0,0,0,0,0,1,rHGG,0,0,1
40,2089,5766,7835,10/19/08,10/20/08,Grade IV,Glioblastoma,31B20,0,0,3,0,0,0.99,f,1.85,1.46,1.31,1.02,1.3900000,0.7500000,1.1700000,1.4700000,1.5400000,1.5300000,NA,NA,NA,NA,NA,NA,NA,perf_align-nonexist,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,1,0,BV hyalinized,NA,NA,NA,NA,1,1,0,1,0,0,1,0,46.15,15.38,0.00,61.53,0,1,1,0,1,1,TxE,0,0,1
